In [74]:
import numpy as np 
from PIL import Image 
from tqdm import tqdm
import os
import pandas as pd 
from sklearn.model_selection import train_test_split

In [53]:
df = pd.read_csv('final_tagging.csv')
df.head()

,filename,file_url,height,width,channel,class,intensity,xmin,xmax,ymin,ymax,polygon,owner
0,IMG_1162.JPG,/Users/smruti.chourasia1/Desktop/Electric/scra...,3024,4032,3,Scratch,Low,1923.3,2197.9,1444.3,1647.5,"[1937.6, 1444.3, 1959, 1572.6, 2176.5, 1590.5,...",smruti
1,IMG_1162.JPG,/Users/smruti.chourasia1/Desktop/Electric/scra...,3024,4032,3,Scratch,Low,443.4,1727.2,1847.2,2007.7,"[468.4, 1879.3, 828.6, 1847.2, 832.1, 1882.9, ...",smruti
2,IMG_1162.JPG,/Users/smruti.chourasia1/Desktop/Electric/scra...,3024,4032,3,Scratch,Low,265.1,561.1,1183.9,1401.5,"[500.5, 1194.6, 265.1, 1372.9, 293.6, 1401.5, ...",smruti
3,IMG_1162.JPG,/Users/smruti.chourasia1/Desktop/Electric/scra...,3024,4032,3,Scratch,Low,1039.0,1249.4,688.3,830.9,"[1210.1, 688.3, 1053.2, 756, 1039, 830.9, 1249...",smruti
4,IMG_1162.JPG,/Users/smruti.chourasia1/Desktop/Electric/scra...,3024,4032,3,Scratch,Low,1031.8,1334.9,955.7,1126.9,"[1228, 962.8, 1031.8, 1059.1, 1053.2, 1126.9, ...",smruti


In [54]:
df['bbox'] =df[['xmin', 'ymin', 'xmax' , 'ymax' , 'width', 'height']].values.tolist()

In [55]:
df = df[['filename', 'bbox']]

In [56]:
df.head()

,filename,bbox
0,IMG_1162.JPG,"[1923.3, 1444.3, 2197.9, 1647.5, 4032.0, 3024.0]"
1,IMG_1162.JPG,"[443.4, 1847.2, 1727.2, 2007.7, 4032.0, 3024.0]"
2,IMG_1162.JPG,"[265.1, 1183.9, 561.1, 1401.5, 4032.0, 3024.0]"
3,IMG_1162.JPG,"[1039.0, 688.3, 1249.4, 830.9, 4032.0, 3024.0]"
4,IMG_1162.JPG,"[1031.8, 955.7, 1334.9, 1126.9, 4032.0, 3024.0]"


In [57]:
df = df.groupby('filename')['bbox'].apply(list).reset_index(name ='bboxes')

In [59]:
df

,filename,bboxes
0,IMG_1087.JPG,"[[2008.9, 2760.1, 2444.0, 2860.0, 4032.0, 3024..."
1,IMG_1091.JPG,"[[3528.1, 1959.8, 3683.8, 2277.8, 4032.0, 3024..."
2,IMG_1092.JPG,"[[0.0, 999.4, 4032.0, 2342.6, 4032.0, 3024.0]]"
3,IMG_1093.JPG,"[[1210.1, 1098.3, 3239.2, 2018.4, 4032.0, 3024..."
4,IMG_1094.JPG,"[[1512.0, 2943.2, 1635.6, 3057.3, 3024.0, 4032..."
...,...,...
386,IMG_6588.JPG,"[[1241.8, 2000.4, 1896.5, 2384.9, 3024.0, 4032..."
387,IMG_6589.JPG,"[[1023.6, 722.2, 1111.9, 898.9, 3024.0, 4032.0..."
388,IMG_6590.JPG,"[[1408.1, 2016.0, 1673.1, 2416.1, 3024.0, 4032..."
389,IMG_6591.JPG,"[[1080.7, 1751.0, 2104.3, 2431.7, 3024.0, 4032..."


In [60]:
df_train , df_valid = train_test_split(df , test_size = 0.1 , random_state = 42, shuffle= True)

In [63]:
df_train =df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

In [77]:
def preprocess(data, data_type ='train'):
    OUTPUT_PATH = 'dataset'
    for _ , row in tqdm(data.iterrows(), total = len(data)):
        image_name = row['filename']
        boxes = row['bboxes']
        yolo_data = []
        for bbox in boxes:
            width = bbox[4]
            height = bbox[5]
            xmin = (1024 *bbox[0])/width
            ymin = (1024*bbox[1])/height
            xmax = (1024*bbox[2])/width
            ymax = (1024*bbox[3])/height

            w = xmax - xmin 
            h = ymax - ymin 

            x_center = xmin+w/2
            y_center = ymin+h/2

            x_center /= 1024
            y_center /= 1024 
            w /= 1024 
            h /= 1024

            yolo_data.append([0 , x_center , y_center , w, h])
        yolo_data = np.array(yolo_data)
        label_path = image_name[:-4]
        img = Image.open(os.path.join('image', image_name))
        img = img.resize((1024 , 1024))
        np.savetxt(
            os.path.join(OUTPUT_PATH , f"labels/{data_type}/{label_path}.txt"),
            yolo_data,
            fmt = ["%d" , "%f" , "%f" , "%f" , "%f"]
        )

        filename = os.path.join(OUTPUT_PATH , f"images/{data_type}/{image_name}")
        img.save(filename)


In [78]:
preprocess(df_train)

100%|██████████| 351/351 [03:18<00:00,  1.77it/s]


In [79]:
preprocess(df_valid , 'validation')

100%|██████████| 40/40 [00:08<00:00,  4.88it/s]
